# H&M LASSO demo (Colab-ready)
This notebook works with either:
1. your GitHub CSV (`HandMSales_fullspan.csv`), **or**
2. a CSV you upload that has `month_start`, `sales_channel_id`, etc.

It will try to detect which schema you have.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print("[INFO] imports OK")

In [ ]:
# @title Load data (from GitHub or upload)
# If you're in Colab and want to upload your own file, uncomment the upload block below.

GITHUB_URL = "https://raw.githubusercontent.com/ucla-anderson-SSAI/SSAI/main/HandMSales_fullspan.csv"

try:
    df = pd.read_csv(GITHUB_URL)
    print("[INFO] Loaded from GitHub:", GITHUB_URL)
except Exception as e:
    raise RuntimeError("Could not load from GitHub. Upload a CSV instead.") from e

# --- SCHEMA FIXES ---
# The user's raw file sometimes uses `month_start` instead of `month_ts`.
if "month_start" in df.columns and "month_ts" not in df.columns:
    df["month_ts"] = pd.to_datetime(df["month_start"], errors="coerce")
elif "month_ts" in df.columns:
    # GitHub version
    df["month_ts"] = pd.to_datetime(df["month_ts"], errors="coerce")
else:
    raise ValueError("Could not find a date column (`month_ts` or `month_start`).")

# If we don't have sales_channel_id (GitHub file), create a dummy 1 so later code works
if "sales_channel_id" not in df.columns:
    df["sales_channel_id"] = 1

# common sort
df = df.sort_values(["article_id", "sales_channel_id", "month_ts"]).reset_index(drop=True)
df["month_num"] = df["month_ts"].dt.month

print("[INFO] df shape:", df.shape)
print("[INFO] columns:", df.columns.tolist()[:20], "...")

In [ ]:
# @title Filter to one product_type_name
SELECTED_PRODUCT_TYPE = "Vest top"  # <-- change here
if "product_type_name" not in df.columns:
    raise ValueError("Your CSV must have a `product_type_name` column.")

df_sub = df[df["product_type_name"] == SELECTED_PRODUCT_TYPE].copy()
if df_sub.empty:
    raise ValueError(f"No rows found for product_type_name == {SELECTED_PRODUCT_TYPE!r}")

print(f"[INFO] filtered to {SELECTED_PRODUCT_TYPE!r} → {df_sub.shape}")
print("[INFO] unique articles:", df_sub["article_id"].nunique())

In [ ]:
# @title Feature engineering (lags, rolling mean, price_change)
# group by article + channel because the user's raw file is that granular
df_sub["lag_m1"] = df_sub.groupby(["article_id","sales_channel_id"])["demand"].shift(1)
df_sub["lag_m2"] = df_sub.groupby(["article_id","sales_channel_id"])["demand"].shift(2)
df_sub["lag_m3"] = df_sub.groupby(["article_id","sales_channel_id"])["demand"].shift(3)

df_sub["ma_3"] = (
    df_sub.groupby(["article_id","sales_channel_id"])["demand"]
          .shift(1).rolling(3, min_periods=1).mean()
          .reset_index(level=[0,1], drop=True)
)

# price_change as pct change of mean_price
df_sub["price_change"] = (
    df_sub.groupby(["article_id","sales_channel_id"])["mean_price"]
          .pct_change()
)

fe_cols = ["lag_m1","lag_m2","lag_m3","ma_3","price_change"]
df_sub[fe_cols] = df_sub[fe_cols].fillna(0)

# time-based split
months_sub = np.sort(df_sub["month_ts"].dropna().unique())
train = df_sub[df_sub["month_ts"] < months_sub[-1]].copy()
test  = df_sub[df_sub["month_ts"] == months_sub[-1]].copy()

print("[INFO] train rows:", len(train), "| test rows:", len(test))

In [ ]:
# @title Fit 3 LASSO models

# ---------- Model A ----------
modelA_num = ["mean_price"]
modelA_cat = ["index_group_name", "month_num", "sales_channel_id"]

train_A = pd.get_dummies(train[modelA_num + modelA_cat], drop_first=False)
test_A  = pd.get_dummies(test[modelA_num + modelA_cat], drop_first=False)
train_A, test_A = train_A.align(test_A, join="left", axis=1, fill_value=0)

scaler_A = StandardScaler()
train_A[modelA_num] = scaler_A.fit_transform(train_A[modelA_num])
test_A[modelA_num]  = scaler_A.transform(test_A[modelA_num])

X_tr_A = train_A.to_numpy()
X_te_A = test_A.to_numpy()
y_tr = train["demand"].to_numpy()
y_te = test["demand"].to_numpy()

model_A = LassoCV(cv=3, max_iter=5000, random_state=0, n_jobs=-1).fit(X_tr_A, y_tr)
pred_A = model_A.predict(X_te_A)

r2_A = r2_score(y_te, pred_A)
rmse_A = mean_squared_error(y_te, pred_A) ** 0.5
mae_A = mean_absolute_error(y_te, pred_A)
print(f"Model A — R²={r2_A:.3f}, RMSE={rmse_A:.2f}, MAE={mae_A:.2f}")

# ---------- Model B ----------
modelB_num = ["mean_price","price_change","lag_m1","lag_m2","lag_m3","ma_3"]
modelB_cat = ["index_group_name", "month_num", "sales_channel_id"]

train_B = pd.get_dummies(train[modelB_num + modelB_cat], drop_first=False)
test_B  = pd.get_dummies(test[modelB_num + modelB_cat], drop_first=False)
train_B, test_B = train_B.align(test_B, join="left", axis=1, fill_value=0)

scaler_B = StandardScaler()
train_B[modelB_num] = scaler_B.fit_transform(train_B[modelB_num])
test_B[modelB_num]  = scaler_B.transform(test_B[modelB_num])

X_tr_B = train_B.to_numpy()
X_te_B = test_B.to_numpy()

model_B = LassoCV(cv=3, max_iter=5000, random_state=0, n_jobs=-1).fit(X_tr_B, y_tr)
pred_B = model_B.predict(X_te_B)

r2_B = r2_score(y_te, pred_B)
rmse_B = mean_squared_error(y_te, pred_B) ** 0.5
mae_B = mean_absolute_error(y_te, pred_B)
print(f"Model B — R²={r2_B:.3f}, RMSE={rmse_B:.2f}, MAE={mae_B:.2f}")

# ---------- Model C ----------
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly_tr = poly.fit_transform(train_B[modelB_num])
poly_te = poly.transform(test_B[modelB_num])

X_tr_C = np.hstack([train_B.drop(columns=modelB_num).to_numpy(), poly_tr])
X_te_C = np.hstack([test_B.drop(columns=modelB_num).to_numpy(),  poly_te])

model_C = LassoCV(cv=3, max_iter=6000, random_state=0, n_jobs=-1).fit(X_tr_C, y_tr)
pred_C = model_C.predict(X_te_C)

r2_C = r2_score(y_te, pred_C)
rmse_C = mean_squared_error(y_te, pred_C) ** 0.5
mae_C = mean_absolute_error(y_te, pred_C)
print(f"Model C — R²={r2_C:.3f}, RMSE={rmse_C:.2f}, MAE={mae_C:.2f}")

In [ ]:
# @title Summary
summary = pd.DataFrame({
    "model": [
        "Model A: baseline + month + channel",
        "Model B: + lags / MA / price_change + month + channel",
        "Model C: + interactions + month + channel",
    ],
    "R2": [r2_A, r2_B, r2_C],
    "RMSE": [rmse_A, rmse_B, rmse_C],
    "MAE": [mae_A, mae_B, mae_C],
})
summary